[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/examples/pinecone-and-scaling.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/examples/pinecone-and-scaling.ipynb)

# Scaling to Many Routes and Using Pinecone

Semantic router can be used with many hundreds, thousands, or even more routes. At very large scales it can be useful to use a vector database to store and search though your route vector space. Although we do not demonstrate _very large_ scale in this notebook, we will demonstrate more routes than usual and we will also see how to use the `PineconeIndex` for potential scalability and route persistence beyond our local machines.

## Installing the Library

In [ ]:
# !pip install -qU \
#     "semantic-router[local, pinecone]==0.0.22" \
#     datasets==2.17.0

## Downloading Routes

In [6]:
from datasets import load_dataset

data = load_dataset("aurelio-ai/generic-routes", split="train")
data

Dataset({
    features: ['name', 'utterances', 'description', 'function_schema', 'llm', 'score_threshold'],
    num_rows: 50
})

Each row in this dataset is a single route:

In [7]:
data[0]

{'name': 'politics',
 'utterances': ["isn't politics the best thing ever",
  "why don't you tell me about your political opinions",
  "don't you just love the presidentdon't you just hate the president",
  "they're going to destroy this country!",
  'they will save the country!'],
 'description': None,
 'function_schema': None,
 'llm': None,
 'score_threshold': 0.82}

We transform these into `Route` objects like so:

In [8]:
from semantic_router import Route

routes = [Route(**data[i]) for i in range(len(data))]
routes[0]

Route(name='politics', utterances=["isn't politics the best thing ever", "why don't you tell me about your political opinions", "don't you just love the presidentdon't you just hate the president", "they're going to destroy this country!", 'they will save the country!'], description=None, function_schema=None, llm=None, score_threshold=0.82)

Next we initialize an `encoder`. We will use a simple `HuggingFaceEncoder`, we can also use popular encoder APIs like `CohereEncoder` and `OpenAIEncoder`.

In [9]:
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder()

Now we initialize our `PineconeIndex`, all it requires is a [Pinecone API key](https://app.pinecone.io) (you do need to be using Pinecone Serverless).

In [2]:
import os
from getpass import getpass
from semantic_router.index.pinecone import PineconeIndex

os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)

index = PineconeIndex(index_name="index", namespace="namespace")

In [11]:
from semantic_router import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=index)

2024-04-15 01:56:59 INFO semantic_router.utils.logger local


We run the solely static routes layer:

In [12]:
rl("how's the weather today?").name

'chitchat'

_If you see a warning about no classification being found, wait a moment and run the above cell again._

## Loading Index From Previous Initialization

Because we're using Pinecone our route index can now persist / be access from different locations by simply connecting to the pre-existing index, by default this index uses the identifier `"semantic-router--index"` — this is the index we'll be loading here, but we can change the name via the `index_name` parameter if preferred.

First, let's delete our old route layer, `index`, and `routes`.

In [ ]:
del rl, index, routes

Let's load our index first. As mentioned, `"index"` is the default index name and we are passing `"namespace"` as namespace name for the pinecone object.

In [3]:
index = PineconeIndex(index_name="index", namespace="namespace")

We load the pre-existing routes from this index like so:

In [2]:
index.get_routes()

[('cybersecurity_best_practices',
  'introduction to ethical hacking for developers'),
 ('coding_standards_and_conventions', 'JavaScript coding conventions'),
 ('chitchat', 'lovely weather today'),
 ('cybersecurity_best_practices', 'securing your web applications'),
 ('gaming_and_esports', 'what are the popular games right now?'),
 ('compliments', 'say something nice about me'),
 ('jokes', 'know any good jokes?'),
 ('data_structures_and_algorithms', 'algorithms every developer should know'),
 ('data_structures_and_algorithms', 'basic data structures for beginners'),
 ('interview_preparation', 'how to prepare for a coding interview'),
 ('career_advice', 'suggest some career development tips'),
 ('creative_writing_and_literature', 'how can I improve my writing skills?'),
 ('art_and_culture', 'tell me about your favorite artist'),
 ('language_learning', 'suggest ways to learn a new language'),
 ('fitness_tips', 'suggest a workout routine'),
 ('mental_health_support', 'what are ways to imp

We will transform these into a dictionary format that we can use to initialize our `Route` objects.

In [5]:
routes_dict = {}
for route, utterance in index.get_routes():
    if route not in routes_dict:
        routes_dict[route] = []
    routes_dict[route].append(utterance)

In [6]:
routes_dict

{'fitness_tips': ['give me a fitness tip',
  'suggest a workout routine',
  'how can I stay active at home?'],
 'language_learning': ['how can I improve my Spanish?',
  'what are some effective language learning techniques?',
  'suggest ways to learn a new language'],
 'frameworks_and_libraries': ['introduction to Django for web development',
  "what's the difference between React and Angular?",
  'best Python libraries for data analysis'],
 'book_recommendations': ["what's your favorite book?",
  'suggest a good book to read',
  'I need a book recommendation'],
 'coding_standards_and_conventions': ['JavaScript coding conventions',
  'maintaining consistency in codebase',
  'why coding standards matter'],
 'historical_events': ['share an interesting piece of medieval history',
  'who was a notable figure in ancient history?',
  'tell me about a significant historical event'],
 'ethical_considerations_in_tech': ['ethical hacking and its importance',
  'the role of ethics in artificial i

Now we transform these into a list of `Route` objects.

In [13]:
routes = [
    Route(name=route, utterances=utterances)
    for route, utterances in routes_dict.items()
]
routes[0]

Route(name='food_and_recipes', utterances=["what's your favorite food?", 'suggest a recipe for dinner', 'tell me about a dish from your country'], description=None, function_schema=None, llm=None, score_threshold=None)

Now we reinitialize our `RouteLayer`:

In [14]:
from semantic_router import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=index)

2024-04-15 01:57:19 INFO semantic_router.utils.logger local


And test it again:

In [15]:
rl("say something to make me laugh").name

'jokes'

In [16]:
rl("tell me something amusing").name

'jokes'

In [17]:
rl("it's raining cats and dogs today").name

'chitchat'

In [7]:
# delete index
index.delete_index()

Perfect, our routes loaded from our `PineconeIndex` are working as expected! As mentioned, we can use the `PineconeIndex` for persistance and high scale use-cases, for example where we might have hundreds of thousands of utterances, or even millions.

---